## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-08-22-52-43 +0000,nyt,Why Cambodia Handed Over a Man Accused of Stea...,https://www.nytimes.com/2026/01/08/world/asia/...
1,2026-01-08-22-49-34 +0000,nyt,Live Updates: Senate Advances Measure to Curb ...,https://www.nytimes.com/live/2026/01/08/world/...
2,2026-01-08-22-48-00 +0000,wsj,Opinion | The Trump-Warren Housing Condominium,https://www.wsj.com/opinion/donald-trump-housi...
3,2026-01-08-22-47-17 +0000,nyt,"Rocío San Miguel, Venezuelan Political Prisone...",https://www.nytimes.com/2026/01/08/world/ameri...
4,2026-01-08-22-46-21 +0000,nyt,Democratic lawmakers ask for an investigation ...,https://www.nytimes.com/live/2026/01/08/us/tru...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2331/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
13,trump,89
16,venezuela,37
118,ice,32
65,shooting,25
82,new,22


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
113,2026-01-08-20-10-00 +0000,wsj,President Trump lashed out at five GOP senator...,https://www.wsj.com/politics/policy/senate-vot...,198
35,2026-01-08-22-07-42 +0000,nyt,We Pressed Trump on His Conclusion About the I...,https://www.nytimes.com/2026/01/08/us/politics...,176
13,2026-01-08-22-40-00 +0000,wsj,The architect of a new White House ballroom to...,https://www.wsj.com/politics/policy/white-hous...,173
46,2026-01-08-21-48-50 +0000,nyt,"Trump Addresses Venezuela, Greenland and Presi...",https://www.nytimes.com/2026/01/08/us/politics...,172
126,2026-01-08-19-28-52 +0000,nyt,Videos Contradict Trump Administration Account...,https://www.nytimes.com/video/us/1000000106310...,170


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
113,198,2026-01-08-20-10-00 +0000,wsj,President Trump lashed out at five GOP senator...,https://www.wsj.com/politics/policy/senate-vot...
45,105,2026-01-08-21-49-52 +0000,nypost,Renee Nicole Good was Minneapolis ‘ICE Watch’ ...,https://nypost.com/2026/01/08/us-news/renee-ni...
13,74,2026-01-08-22-40-00 +0000,wsj,The architect of a new White House ballroom to...,https://www.wsj.com/politics/policy/white-hous...
189,51,2026-01-08-16-59-04 +0000,nypost,Wealth tax threat prompts at least six billion...,https://nypost.com/2026/01/08/business/wealth-...
85,43,2026-01-08-20-50-37 +0000,startribune,Reusse: Eric Musselman will coach for first ti...,https://www.startribune.com/patrick-reusse-col...
259,43,2026-01-08-12-01-00 +0000,wsj,China Warns AI Startups Seeking to Emulate Met...,https://www.wsj.com/tech/china-warns-ai-startu...
127,42,2026-01-08-19-14-24 +0000,nypost,Protests erupt in Iranian capital after exiled...,https://nypost.com/2026/01/08/world-news/prote...
353,40,2026-01-08-03-28-00 +0000,wsj,Trump Team Works Up Sweeping Plan to Control V...,https://www.wsj.com/business/energy-oil/trump-...
53,34,2026-01-08-21-37-00 +0000,wsj,President Trump demonstrated the tight grip he...,https://www.wsj.com/politics/policy/trump-flex...
380,32,2026-01-08-00-04-09 +0000,wapo,Europe draws red line on Greenland after a yea...,https://www.washingtonpost.com/world/2026/01/0...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
